In [231]:
import json
from lxml import objectify
from selenium import webdriver
from bs4 import BeautifulSoup

import urllib
from urllib.request import urlopen
from urllib.error import HTTPError
from datetime import datetime
from datetime import timedelta
import time
import pandas
import numpy
import re

## Links and params

In [2]:
## LTA
headers = { 'AccountKey' : 'LBdfS4+RSEi4witEa6RjjQ==',
            'UniqueUserID' : '812eb03f-f3a1-421c-96de-069c59844bbb',
            'accept' : 'application/json'}
uri = 'http://datamall2.mytransport.sg/'
path_TSB = 'ltaodataservice/TrafficSpeedBands'
path_TI = 'ltaodataservice/TrafficIncidents'
path_FTL = 'ltaodataservice/FaultyTrafficLights'
path_ETT = 'ltaodataservice/EstTravelTimes'
path_RO = 'ltaodataservice/RoadOpenings'
path_RW = 'ltaodataservice/RoadWorks'
## NEA links and auth params
key = '781CF461BB6606ADC767F3B357E848ED4A6709A0B39C63D6'
urlNowcast = 'http://api.nea.gov.sg/api/WebAPI/?dataset=2hr_nowcast&keyref=' + key
urlWarning = 'http://api.nea.gov.sg/api/WebAPI/?dataset=heavy_rain_warning&keyref=' + key
## Meteorological Service Singapore Rainfall observations
urlRainfall = 'http://www.weather.gov.sg/weather-currentobservations-rainfall'

### Test LTA data EPN

In [194]:
# Traffic Speed Band data - updates every 5 minutes
request = urllib.request.Request(uri + path_TSB, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfTSB = pandas.DataFrame(messages)

In [203]:
#dfTSB = dfTSB.set_index('LinkID')
#dfTSB.index.name = None
dfTSB.head()

,Location,MaximumSpeed,MinimumSpeed,RoadCategory,RoadName,SpeedBand
103098943,1.3805895432850994 103.74201730473243 1.380040...,59,40,D,CHOA CHU KANG AVENUE 3,3
103076063,1.399027473240379 103.7553182996927 1.39866097...,59,40,B,WOODLANDS ROAD,3
103030883,1.3742439017982357 103.88529975827204 1.373869...,39,20,D,HOUGANG AVENUE 4,2
104003992,1.3163287378822295 103.76133268167833 1.315260...,59,40,F,CLEMENTI AVENUE 6,3
103110036,1.333534171783063 103.88445104036495 1.3344802...,39,20,E,HARVEY ROAD,2


In [202]:
#dfTSB2 = dfTSB.set_index('LinkID')
#dfTSB2.index.name = None
dfTSB2.head()

,Location,MaximumSpeed,MinimumSpeed,RoadCategory,RoadName,SpeedBand
103098943,1.3805895432850994 103.74201730473243 1.380040...,39,20,D,CHOA CHU KANG AVENUE 3,2
103076063,1.399027473240379 103.7553182996927 1.39866097...,59,40,B,WOODLANDS ROAD,3
103030883,1.3742439017982357 103.88529975827204 1.373869...,39,20,D,HOUGANG AVENUE 4,2
104003992,1.3163287378822295 103.76133268167833 1.315260...,59,40,F,CLEMENTI AVENUE 6,3
103110036,1.333534171783063 103.88445104036495 1.3344802...,39,20,E,HARVEY ROAD,2


In [240]:
neRows = (dfTSB != dfTSB2).any(1)
neRows
ne_stacked = (dfTSB != dfTSB2).stack()
ne_stacked
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
changed
diff_loc = numpy.where(dfTSB != dfTSB2)
changed_from = dfTSB.values[diff_loc]
changed_to = dfTSB2.values[diff_loc]
diff = pandas.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)
diff.head()

from  to
id        col                  
103098943 MaximumSpeed   59  39
          MinimumSpeed   40  20
          SpeedBand       3   2
103070015 MaximumSpeed   59  39
          MinimumSpeed   40  20

In [279]:
for id, row in diff.iterrows():
    if (id[1] == 'SpeedBand'):
        print(row['from'], row['to']) 

3 2
3 2
3 4
2 1
3 2
2 1


In [218]:
df1 = df.copy()
if(df.equals(df1)):
    print('same')
if not (dfTSB.equals(dfTSB2)):
    print('diff')

same
diff


In [9]:
# ESTIMATED TRAVEL TIMES data - updates every 5 minutes
request = urllib.request.Request(uri + path_ETT, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfETT = pandas.DataFrame(messages)

In [18]:
dfETT.head(2)

,Direction,EndPoint,EstTime,FarEndPoint,Name,StartPoint
0,1,TELOK BLANGAH RD,2,TUAS CHECKPOINT,AYE,AYE/MCE INTERCHANGE
1,1,LOWER DELTA RD,1,TUAS CHECKPOINT,AYE,TELOK BLANGAH RD


In [13]:
# collect Traffic Incidents data - updates every 2 minutes
request = urllib.request.Request(uri + path_TI, headers=headers)
try:
    response = urlopen(request).read().decode('utf-8')
except HTTPError as e:
    print (e)
else:
    jsonObj = json.loads(response)
    messages = jsonObj.get("value")
    dfTI = pandas.DataFrame(messages)

In [19]:
dfTI.head(2)

,Latitude,Longitude,Message,Type
0,1.392957,103.824566,(8/11)22:09 Vehicle breakdown on SLE (towards ...,Vehicle breakdown
1,1.305273,103.860494,(8/11)22:05 Roadworks on Victoria Street (towa...,Roadwork


In [156]:
# collect Nowcast data - updates every 30 mins
request = urllib.request.Request(urlNowcast)
try:
    parsed = objectify.parse(urlopen(request))
except HTTPError as e:
    print (e)
else:
    parsed = objectify.parse(urlopen(request))
    root = parsed.getroot()
    dfNowcast = pandas.DataFrame(columns=['Forecast', 'Latitude', 'Longitude', 'Name'])
    for area in root.iter('area'):
        dfNowcast = dfNowcast.append({'Forecast':area.get('forecast'),'Latitude':area.get('lat'),'Longitude':area.get('lon'),'Name':area.get('name')}, ignore_index=True)

In [157]:
dfNowcast.head(2)

,Forecast,Latitude,Longitude,Name
0,TL,1.37500000,103.83900000,Ang Mo Kio
1,TL,1.32100000,103.92400000,Bedok


In [185]:
dfNowc = dfNowcast.set_index('Name')
dfNowc.index.name = None
print(dfNowc.head())

            Forecast    Latitude     Longitude
Ang Mo Kio        TL  1.37500000  103.83900000
Bedok             TL  1.32100000  103.92400000
Bishan            TL  1.35077200  103.83900000
Boon Lay          SH  1.30400000  103.70100000
Bukit Batok       SH  1.35300000  103.75400000


In [183]:
# collect Heavy Rain Warning data - updates every hour
request = urllib.request.Request(urlWarning)
try:
    parsed = objectify.parse(urlopen(request))
except HTTPError as e:
    print (e)
else:
    parsed = objectify.parse(urlopen(request))
    root = parsed.getroot()
    HRWmsg = root.item.warning.text
    HRWmsg = HRWmsg.strip()
    HRWtime = root.item.issue_datentime.text
    HRWtime = HRWtime.strip()

In [184]:
if('NIL' in HRWmsg):
    print('No warnings')
else:
    print(HRWmsg)

No warnings


In [173]:
# collect Rainfall data - updates every 10 mins
driver = webdriver.PhantomJS(executable_path='../bin/phantomjs')
try:
    driver.get(urlRainfall)
except HTTPError as e:
    print (e)
else:
    # wait for page load
    time.sleep(2)
    html30mins = driver.page_source
    bsObj = BeautifulSoup(html30mins,"lxml")
    # Get data timestamp
    element = bsObj.find("img", {"id":"basemap"})
    pattern = re.compile('[0-9]+:[0-9]+')
    rainfallTimestamp = pattern.findall(element.attrs['src'])[0]
    # load 30 mins rainfall data
    dataset30mins =  bsObj.findAll("",{"class":"sgr"})
    dfRainfall = pandas.DataFrame(columns=['StationId', 'rain30mins'])
    for data in dataset30mins:
        dfRainfall = dfRainfall.append({'StationId':data.get('id'), 'rain30mins':data.get_text()},ignore_index=True)

In [174]:
dfRainfall.head(2)

,StationId,rain30mins
0,S07,0
1,S08,0


In [175]:
dfRain = dfRainfall.set_index('StationId')
dfRain.index.name = None
dfRain.ix['S113']
dfRain.head()

,rain30mins
S07,0
S08,0
S101,0
S104,0
S106,0
